In [150]:
from web3 import Web3
from web3.gas_strategies.rpc import rpc_gas_price_strategy
from uniswap import Uniswap
import time
from dotenv import load_dotenv
import os
import json

In [151]:
class TokenTrader:
    def __init__(self, rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds, lot_size_tokens, min_sell_amount_tokens):
        self.rpc_url = rpc_url
        self.wallet_address = wallet_address
        self.private_key = private_key
        self.token_address = token_address
        self.usdt_address = usdt_address
        self.eth_address = eth_address
        self.price_thresholds = price_thresholds
        self.lot_size_tokens = lot_size_tokens
        self.min_sell_amount_tokens = min_sell_amount_tokens
        
        self.web3 = Web3(Web3.HTTPProvider(self.rpc_url))
        self.web3.eth.set_gas_price_strategy(rpc_gas_price_strategy)
        self.uniswap = Uniswap(address=self.wallet_address, private_key=self.private_key, version=2, web3=self.web3, provider=self.rpc_url)
        
        self.eth_decimals = self.get_token_decimals(self.eth_address)
        self.usdt_decimals =self.get_token_decimals(self.usdt_address)
        self.token_decimals = self.get_token_decimals(self.token_address)
        self.lot_size = int(self.lot_size_tokens * 10**self.token_decimals)
        self.min_sell_amount = int(self.min_sell_amount_tokens * 10**self.token_decimals)

        self.state_file = 'trader_state.txt'
        self.sold_lots, self.sold_tokens = self.load_state()

    def load_state(self):
        if os.path.exists(self.state_file):
            with open(self.state_file, 'r') as file:
                state = json.load(file)
                sold_lots = int(state['sold_lots'])
                sold_tokens = [int(token) for token in state['sold_tokens']]
        else:
            sold_lots = 0
            sold_tokens = [0] * len(self.price_thresholds)
        return sold_lots, sold_tokens

    def save_state(self):
        state = {
            'sold_lots': str(self.sold_lots),
            'sold_tokens': [str(token) for token in self.sold_tokens] 
        }
        with open(self.state_file, 'w') as file:
            json.dump(state, file)

    def get_token_decimals(self, token_address):
        token_contract = self.uniswap.get_token(token_address)
        return token_contract.decimals

    def get_token_balance(self, token_address):
        return self.uniswap.get_token_balance(token_address)
    
    def get_price_from_pool(self, input_token, output_token, decimals):
        try:
            price = self.uniswap.get_price_input(input_token, output_token, 10**decimals)
            return price
        except Exception as e:
            print(f"An error occurred while fetching price: {e}")
            return None

    def get_token_price_in_usdt(self):
        token_price_in_eth = self.get_price_from_pool(self.token_address, self.eth_address, self.token_decimals)
        eth_price_in_usdt = self.get_price_from_pool(self.eth_address, self.usdt_address, self.eth_decimals)
        if token_price_in_eth is not None and eth_price_in_usdt is not None:
            return (token_price_in_eth / 10**self.eth_decimals) * (eth_price_in_usdt / 10**self.usdt_decimals)
        return None

    def sell_token_for_base(self, token_address, base_address, amount):
        tx = self.uniswap.make_trade(token_address, base_address, int(amount))
        txid = tx.hex()
        print(f"Esperando por confirmación de la transacción {txid}")
        self.web3.eth.wait_for_transaction_receipt(tx)
        return txid

    def trade(self):
        while self.sold_lots < len(self.price_thresholds):
            try:
                balance = int(self.get_token_balance(self.token_address))
                print(f"Balance actual: {balance / 10**self.token_decimals} tokens")
                price_in_usdt = self.get_token_price_in_usdt()
                print(f"Precio actual: {price_in_usdt} USDT")
        
                # Actualiza el lote actual basado en el precio
                while self.sold_lots < len(self.price_thresholds) and price_in_usdt >= self.price_thresholds[self.sold_lots]:
                    self.sold_lots += 1
                self.sold_lots -= 1 

                # Determinar cantidad total a vender basada en los lotes pendientes
                total_to_sell = 0
                for i in range(self.sold_lots + 1):
                    total_to_sell += self.lot_size - self.sold_tokens[i]

                amount_to_sell = min(balance, total_to_sell)

                if amount_to_sell >= self.min_sell_amount:
                    print(f"Vendiendo {amount_to_sell / 10**self.token_decimals} tokens por {self.eth_address} a {price_in_usdt} USDT")
                    tx_receipt = self.sell_token_for_base(self.token_address, self.eth_address, amount_to_sell)
                    print(f"Transacción confirmada: {tx_receipt}")

                    # Actualizar estado según lo vendido
                    sold = amount_to_sell
                    for i in range(self.sold_lots + 1):
                        if sold <= 0:
                            break
                        sell_from_lot = min(sold, self.lot_size - self.sold_tokens[i])
                        self.sold_tokens[i] += sell_from_lot
                        sold -= sell_from_lot

                    self.save_state()

                    if (self.sold_tokens[self.sold_lots] >= self.lot_size or 
                        abs(self.sold_tokens[self.sold_lots] - self.lot_size) <= self.min_sell_amount):
                        self.sold_lots += 1 

                time.sleep(1)
            except Exception as e:
                print(f"Ha ocurrido un error: {e}")
                time.sleep(1)

In [152]:
if __name__ == "__main__":
    # Cargar variables de entorno
    load_dotenv()
    
    # Configuración
    rpc_url = "https://rpc.ankr.com/eth" 
    wallet_address = os.getenv("WALLET_ADDRESS")
    private_key = os.getenv("PRIVATE_KEY")
    token_address = "0x6982508145454Ce325dDbE47a25d4ec3d2311933"
    usdt_address = "0xdAC17F958D2ee523a2206206994597C13D831ec7"
    eth_address = "0x0000000000000000000000000000000000000000"
    price_thresholds = [0.00001, 0.0000102, 0.0000103, 0.0000104, 0.0000105]
    lot_size_tokens = 23081
    min_sell_amount_tokens = 5000

    trader = TokenTrader(rpc_url, wallet_address, private_key, token_address, usdt_address, eth_address, price_thresholds, lot_size_tokens, min_sell_amount_tokens)
    trader.trade()

Balance actual: 115407.51351236289 tokens
Precio actual: 1.009631971135757e-05 USDT
Vendiendo 23081.0 tokens por 0x0000000000000000000000000000000000000000 a 1.009631971135757e-05 USDT
Esperando por confirmación de la transacción 0x7b4a11fdf89a60854dfcad9dd29e78dff48c204b7c8455e7d520afa435f14d98
Transacción confirmada: 0x7b4a11fdf89a60854dfcad9dd29e78dff48c204b7c8455e7d520afa435f14d98
Balance actual: 115407.51351236289 tokens
Precio actual: 1.0096272072431185e-05 USDT
Balance actual: 92326.51351236289 tokens
Precio actual: 1.0095843818844533e-05 USDT
Balance actual: 92326.51351236289 tokens
Precio actual: 1.0108122363294072e-05 USDT
Balance actual: 115407.51351236289 tokens
Precio actual: 1.0095843818844533e-05 USDT
Balance actual: 115407.51351236289 tokens
Precio actual: 1.0095843818844533e-05 USDT
Balance actual: 115407.51351236289 tokens
Precio actual: 1.009577100861038e-05 USDT
Balance actual: 115407.51351236289 tokens
Precio actual: 1.009577100861038e-05 USDT
Balance actual: 92326

KeyboardInterrupt: 